In [1]:
!pip install lxml

You should consider upgrading via the 'C:\Users\Yazmina\Documents\MASTER AI\SEL\Coursework\SEL_PW3\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import random

# Own libraries
from cbr.cbr import CBR
from src.entity.query import Query
from definitions import CASE_LIBRARY as CASE_LIBRARY_PATH, DATA_PATH
from cbr.case_library import CaseLibrary, ConstraintsBuilder

random.seed(10)

In [3]:
import random
import pickle

# Own libraries
from cbr import CBR
from src.entity.query import Query
from definitions import CASE_LIBRARY as CASE_LIBRARY_PATH, DATA_PATH
from case_library import CaseLibrary, ConstraintsBuilder

random.seed(10)

SyntaxError: invalid syntax (4098029073.py, line 3)

In [ ]:
print(f"The number of instances from the database {len(cocktails)}")
cocktails.head()

In [2]:
query = Query()
query.set_category("ordinary drink")
query.set_glass("old-fashioned glass")
query.set_ingredients(["orange juice"])
query.set_exc_ingredients(["rum", "grenadine"])
query.set_alc_types(["gin"])
query.set_basic_tastes(["salty"])

# TODO: retrieve 5 recipes similar to the query from the case library
# Random recipes for now
CASE_LIBRARY = CaseLibrary(CASE_LIBRARY_PATH)
CONSTRAINT = ConstraintsBuilder(include_category=query.get_category(), include_glass=query.get_glass())
recipes = random.choices(CASE_LIBRARY.findall(CONSTRAINT), k=5)
print("Done")

Done


In [4]:
query = Query()
query.set_category("ordinary drink")
query.set_glass("old-fashioned glass")
query.set_ingredients(["orange juice"])
query.set_exc_ingredients(["rum", "grenadine"])
query.set_alc_types(["gin"])
query.set_basic_tastes(["salty"])

# TODO: retrieve 5 recipes similar to the query from the case library
# Random recipes for now
CASE_LIBRARY = CaseLibrary(CASE_LIBRARY_PATH)
CONSTRAINT = ConstraintsBuilder(include_category=query.get_category(), include_glass=query.get_glass())
recipes = random.choices(CASE_LIBRARY.findall(CONSTRAINT), k=5)

In [3]:
print(f"Ingredients before: {[e.text for e in recipes[0].ingredients.iterchildren()]}")
print(f"Steps before: {[e.text for e in recipes[0].preparation.iterchildren()]}")

cbr = CBR()
cbr.retrieve(query=query, recipes=recipes)
cbr.adapt()
output = cbr.recipe

print(f"Ingredients after: {[e.text for e in output.ingredients.iterchildren()]}")
print(f"Steps after: {[e.text for e in output.preparation.iterchildren()]}")

Ingredients before: ['151 proof rum', 'grenadine', 'kahlua']
Steps before: ['mix ingr2 and ingr0 in glass', 'quickly add ice and pour ingr1 over ice to give ice red tint']
Ingredients after: ['151 proof rum', 'orange juice', 'kahlua', 'gin', 'worcestershire sauce']
Steps after: ['mix ingr2 and ingr0 in glass', 'add ingr4 to taste', 'add 2 oz of ingr3', 'quickly add ice and pour ingr1 over ice to give ice red tint']


In [ ]:
print(f"Ingredients before: {[e.text for e in recipes[0].ingredients.iterchildren()]}")
print(f"Steps before: {[e.text for e in recipes[0].preparation.iterchildren()]}")

cbr = CBR(query, recipes)
cbr.adapt()
output = cbr.recipe

print(f"Ingredients after: {[e.text for e in output.ingredients.iterchildren()]}")
print(f"Steps after: {[e.text for e in output.preparation.iterchildren()]}")

## EVAL

## LEARN